## Testing staircase

In [ ]:
from moisture_rnn import staircase, staircase_2
import numpy as np
from data_funcs import plot_data

In [ ]:
# training data shape   [batch_size, sequence_length, features] 
# also called           [samples,timesteps,features]
# input data size       [trainsteps,features] 

In [ ]:
trainsteps=100
timesteps=5
features=1
outputs=1
x=np.tile(range(trainsteps), (features, 1)).T
y=np.tile(range(trainsteps), (outputs, 1)).T
# print('x =',x)
print('x shape =',x.shape)
# print('y =',y)
print('y shape =',y.shape)

In [ ]:
x_train, y_train = staircase(x,y,timesteps,trainsteps,return_sequences=False, verbose = True)
print('x_train shape =',x_train.shape)
print('y_train shape =',y_train.shape)
print('x_train =',x_train)
print('y_train =',y_train)


In [ ]:
x_train, y_train = staircase_2(x,y,timesteps,return_sequences=False, verbose = True)
print('x_train shape =',x_train.shape)
print('y_train shape =',y_train.shape)
#print('x_train =',x_train)
#print('y_train =',y_train)
for i in range(x_train.shape[0]):
    print('x_train[',i,']:')
    print(x_train[i])

## Testing RNN training on sin cos problem

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, SimpleRNN
from data_funcs import plot_data

# Generate sample time series data (replace with your actual data)
hours = 500
x = 10*(1+np.cos(np.linspace(0, hours, hours)*2*np.pi/24))  # Ensure divisible by batch size and lookback
x = x + 2.0*np.random.randn(*x.shape)
y = np.zeros((hours))
for i in range(1,hours):
    y[i] = y[i-1] + (x[i-1] - y[i-1])/10.
x=np.reshape(x,[-1,1])
y=np.reshape(y,[-1,1])
print('x.shape',x.shape)
print('y.shape',y.shape)

In [ ]:
plot_data({'x':x,'y':y},xlabel='',ylabel='')

In [ ]:
# Create training data with lookback, offset by one, all one batch
x_train, y_train = [], []
timesteps = 20
for i in range(len(x) - timesteps):
    x_train.append(x[i:i+timesteps])  # Create sequences of 5 timesteps
    y_train.append(y[i+timesteps])
x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape input data for RNN
x_train = x_train.reshape(-1, timesteps, 1)  # Print x_train.shape to verify
print(x_train)
print('x_train.shape',x_train.shape)
print('y_train.shape',y_train.shape)

In [ ]:
x_train, y_train = staircase_2(x,y,timesteps,return_sequences=False, verbose = True)

In [ ]:
# Define the stateful RNN model
RNN=SimpleRNN
#RNN=LSTM
#RNN=GRU
batch_size=timesteps  # each time series its own batch
epochs=200

model = Sequential()
model.add(RNN(32, stateful=True, batch_input_shape=(batch_size, timesteps, 1)))  # Adjust batch size if needed
model.add(Dense(1))  # Output layer for single-value prediction
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model (manual state resetting)'print('x_train.shape',x_train.shape)
print('x_train.shape',x_train.shape)
print('y_train.shape',y_train.shape)
if len(x_train.shape) < 4:
    for i in range(epochs):
        model.fit(x_train, y_train, epochs=1, batch_size=batch_size, verbose=1, shuffle=False)
        model.reset_states()
else:
    batches = x_train.shape[0]  
    print('x_train has',batches,'batches')
    for i in range(epochs):
        for j in range(batches):
            print('iteration',i,'batch',j,'size',batch_size)
            model.fit(x_train[j], y_train[j], epochs=1, batch_size=batch_size, verbose=1, shuffle=False)
        model.reset_states()  # at the end of each iteration = epoch
# print('weights',model.get_weights())

# Define the stateless RNN model - to be used on data in a single sequence
model2 = Sequential()
model2.add(RNN(32, stateful=False,input_shape=(None,1))) 
model2.add(Dense(1))  # Output layer for single-value prediction
model2.compile(loss='mean_squared_error', optimizer='adam')

# transfer weights, predict, plot
w=model.get_weights()
model2.set_weights(w)
z = model2.predict(x.reshape((-1,1))) # inout and output need to be shape (-1,1), ie columns
plot_data({'x':x,'y':y,'z':z},xlabel='',ylabel='')

In [ ]:
plot_data({'x':x,'y':y,'z':z},xlabel='',ylabel='')